https://www.youtube.com/watch?v=zy8ChVd_oTM

# Softmax (safe, online)

In [ ]:
# numerical instability

# Block matrix multiplication

# Flash Attention Forward

Parallel hierarchy:

Batch size X Num heads (multihead) X Blocks of Q (Seq_length / block_size)

# Intro to CUDA

In [ ]:
vector adding, kernel, indexingm, block

The basic unit of a GPU is 1 control unit + a bunch of cores/threads (also called a warp)

Since there is only 1 control unit, all threads can only do the same computation

Say we want to do vector addition with a vector of length 64, and our GPU only has 16 threads, we can divide the data into blocks, each with size 2, 4, 8 or 16. This way the GPU can process one or more blocks at a time.

A kernel is how one control unit computes the data?

# Intro to Tensor

Stride, matrix, transpose

Matrices are stored as tensors, which essentially is [1, 2, 3, 4]

Stride is the method for accessing them. For example, a stride of [2,1 ] means starting from the third variable, with a step 1, giving [3, 4] of the matrix.

Other matrix operations such as transpose are also done in the same way[link text](https://)

# Intro to Triton

In [ ]:
# Triton add_vector kernel implementation

import torch
import triton
import triton.language as tl

@triton.jit
def add_kernel(

    # No access to the whole vector,
    # only *Pointer* to the start
    x_ptr,  # first input vector.
    y_ptr,  # second input vector.
    output_ptr,  # output vector.

    n_elements,  # Size of the vector.
    BLOCK_SIZE: tl.constexpr,  # Number of elements each program should process.
):
    # NOTE: `constexpr` so it can be used as a shape value.

    # There are multiple "programs" processing different data. We identify which program we are here:
    pid = tl.program_id(axis=0)  # We use a 1D launch grid so axis is 0.

    # This program will process inputs that are offset from the initial data.
    # For instance, if you had a vector of length 256 and block_size of 64, the programs
    # would each access the elements [0:64, 64:128, 128:192, 192:256].
    # Note that offsets is a list of pointers:
    block_start = pid * BLOCK_SIZE
    offsets = block_start + tl.arange(0, BLOCK_SIZE)  # 2048, 2049, ... 2060, 2061, 2062...

    # Create a mask to guard memory operations against out-of-bounds accesses.
    mask = offsets < n_elements

    # Load x and y from DRAM, masking out any extra elements in case the input is not a multiple of the block size.
    x = tl.load(x_ptr + offsets, mask=mask)
    y = tl.load(y_ptr + offsets, mask=mask)

    output = x + y

    # Write x + y back to DRAM.
    tl.store(output_ptr + offsets, output, mask=mask)


# Flash Attention forward

In [ ]:
# import
import torch
import triton
import triton.language as tl

In [ ]:
# Every operation implemented with pytorch, such as softmax, relu
# is derived from torch.autograd.Function
# Each implementation should contain forward pass and backward pass
class TritonAttention(torch.autograd.Function):

  @staticmethod
  def forward(ctx, Q, K, V, causal, softmax_scale):

    # check size
    HEAD_DIM_Q, HEAD_DIM_K, HEAD_DIM_V = Q.shape[-1], K.shape[-1]
    HEAD_DIM_V = V.shape[-1]

    BATCH_SIZE, NUM_HEADS, SEQ_LEN, HEAD_DIM = Q.shape

    assert HEAD_DIM_Q == HEAD_DIM_K and HEAD_DIM_K == HEAD_DIM_V


